In [6]:
!pip install "sagemaker>=2.31.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade

     |████████████████████████████████| 420 kB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 44.7 MB/s eta 0:00:01
     |████████████████████████████████| 221 kB 22.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 53.9 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 71.6 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 47.2 MB/s eta 0:00:01     |███████████████████████████████▎| 7.0 MB 47.2 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 51.9 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 49 kB 6.9 MB/s  eta 0:00:01
     |████████████████████████████████| 48 kB 724 kB/s  eta 0:00:01
     |████████████████████████████████| 48 kB 5.7 MB/s  eta 0:00:01
  Created wheel for sagemaker: filename=

In [27]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [28]:
data_dir = '../data/bert' # The folder we will use for storing data
os.makedirs(data_dir, exist_ok=True)  # ensure cache directory exists

cache_dir = os.path.join("../cache", "bert")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

# 1. Preparing Data

## 1.1. Loading Dataset

In [30]:
train_df = pickle.load(open('../data/processed/train_df.pkl', 'rb'))
val_df = pickle.load(open('../data/processed/val_df.pkl', 'rb'))
test_df = pickle.load(open('../data/processed/test_df.pkl', 'rb'))

train_df = pd.concat([train_df, val_df]).reset_index(drop = True)
val_df = None

train_df = train_df.rename(columns = {'cleaned_y': 'label', 'all_text': 'sentence'})
# val_df = val_df.rename(columns = {'cleaned_y': 'label', 'all_text': 'sentence'})
test_df = test_df.rename(columns = {'cleaned_y': 'label', 'all_text': 'sentence'})

train_df[['label', 'sentence']].to_csv(os.path.join(data_dir, 'train_bert.csv'), index = False, header = True)
# val_df[['label', 'sentence']].to_csv(os.path.join(data_dir, 'val_bert.csv'), index = False, header = True)
test_df[['label', 'sentence']].to_csv(os.path.join(data_dir, 'test_bert.csv'), index = False, header = True)

In [35]:
train_dataset = load_dataset('csv', data_files=os.path.join(data_dir, 'train_bert.csv'))
test_dataset = load_dataset('csv', data_files=os.path.join(data_dir, 'test_bert.csv'))

Using custom data configuration default-19fb1072c36d318d
Reusing dataset csv (/home/ec2-user/.cache/huggingface/datasets/csv/default-19fb1072c36d318d/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Using custom data configuration default-62865b37610d3fc8


Dataset csv downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/csv/default-62865b37610d3fc8/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [34]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence'],
        num_rows: 45000
    })
})

## 1.2 Process Dataset

In [ ]:
# from datasets import load_dataset
from transformers import AutoTokenizer

# tokenizer used in preprocessing
tokenizer_name = 'distilbert-base-uncased'

# dataset used
dataset_name = 'imdb'

# s3 key prefix for the data
s3_prefix = 'samples/datasets/imdb'

In [38]:
# download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['sentence'], padding='max_length', truncation=True)

In [39]:
# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set format for pytorch
train_dataset =  train_dataset.rename_column("label", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

## 1.3 Uploading Data

In [ ]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [40]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train_bert'
train_dataset.save_to_disk(training_input_path,fs=s3)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test_bert'
test_dataset.save_to_disk(test_input_path,fs=s3)

In [ ]:
# # load dataset
# dataset = load_dataset(dataset_name)

# 2. Create Model

In [52]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'per_device_train_batch_size': 2,
                 'model_name_or_path': 'distilbert-base-uncased'
                 }

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='huggingface_train.py',
        source_dir='../src_bert',
        instance_type='ml.p2.xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.4',
        pytorch_version='1.6',
        py_version='py36',
        hyperparameters = hyperparameters
)

In [53]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.p2.xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.